In [1]:
import pandas as pd
train_val = pd.read_parquet('train_dataset_hackaton2023_train.gzip')
test = pd.read_parquet('hackaton2023_test.gzip')

In [2]:
import random
cust = train_val['customer_id'].unique().tolist()
random.shuffle(cust)
train_cust = cust[:400000]
val_cust = cust[400000:]

In [3]:
val = train_val[train_val['customer_id'].isin(val_cust)]
train = train_val[train_val['customer_id'].isin(train_cust)]

In [4]:
train_X = pd.DataFrame()
val_X = pd.DataFrame()

In [5]:
import numpy as np
train_X['mean_bill'] = train.groupby(['customer_id', 'startdatetime']).revenue.sum().groupby('customer_id').mean()
train_X['mean_time'] = np.array(train.groupby(['customer_id', 'startdatetime']).startdatetime.first().groupby('customer_id').diff().groupby('customer_id').mean().dt.total_seconds())
train_X['mean_rest_size'] = np.array(train.groupby(['customer_id']).ownareaall_sqm.mean())
train_X['last_rest'] = np.array(train.groupby(['customer_id']).format_name.last())
train['discounts'] = train['revenue'] ==1
train_X['discounts'] = (train.groupby(['customer_id']).discounts).sum()

val_X['mean_bill'] = val.groupby(['customer_id', 'startdatetime']).revenue.sum().groupby('customer_id').mean()
val_X['mean_time'] = np.array(val.groupby(['customer_id', 'startdatetime']).startdatetime.first().groupby('customer_id').diff().groupby('customer_id').mean().dt.total_seconds())
val_X['mean_rest_size'] = np.array(val.groupby(['customer_id']).ownareaall_sqm.mean())
val_X['last_rest'] = np.array(val.groupby(['customer_id']).format_name.last())
val['discounts'] = val['revenue'] ==1
val_X['discounts'] = (val.groupby(['customer_id']).discounts).sum()

/tmp/ipykernel_146129/998746970.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['discounts'] = train['revenue'] ==1
/tmp/ipykernel_146129/998746970.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['discounts'] = val['revenue'] ==1


In [6]:
train_Y_cat = np.array(train.groupby('customer_id').buy_post.first())
val_Y_cat = np.array(val.groupby('customer_id').buy_post.first())

train_Y_reg = np.array(train.groupby('customer_id').date_diff_post.first())
val_Y_reg = np.array(val.groupby('customer_id').date_diff_post.first())

train_Y_reg[np.isnan(train_Y_reg)] = 0
val_Y_reg[np.isnan(val_Y_reg)] = 0

In [7]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
model = CatBoostClassifier(iterations=100,
                        learning_rate=0.01,
                        depth=5,
                        l2_leaf_reg = 3,
                        leaf_estimation_iterations = 10,
                        task_type="GPU",
                        verbose=False,
                        cat_features=[3])

model.fit(train_X, train_Y_cat)
f1_score(val_Y_cat, model.predict(val_X))

0.8388629269544562

In [8]:
roc_auc_score(val_Y_cat, model.predict_proba(val_X)[:,1])

0.6268111292040197

In [9]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
model = CatBoostRegressor(iterations=100,
                        learning_rate=0.01,
                        depth=5,
                        l2_leaf_reg = 3,
                        leaf_estimation_iterations = 10,
                        task_type="GPU",
                        verbose=False,
                        cat_features=[3])

model.fit(train_X, train_Y_reg)
mean_squared_error(val_Y_reg, model.predict(val_X), squared = False)

17.00454855710812